In [ ]:
!pip install orq-ai-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.2/481.2 kB 11.0 MB/s eta 0:00:00


# Defining main HR policy agent


**Agent Metadata**

* `role` : description and instructions

* `description` : short description of agent's purpose

* `instructions` : instructions for the agent behaviour

**Configuration parameters**

* `max_iterations` : how many reasoning steps the agent can take. Useful to prevent runaway costs or infinite loops in complex reasoning chains.

* `max_execution_time` : sets a hard timeout in seconds. The agent will gracefully terminate if it exceeds this limit.

* `model`: choose from providers e.g. openai/gpt-4o, claude-3-5-sonnet

* `memory_stores`: adding personal data from conversations (e.g. employee name, preferences, past queries)



In [ ]:
from orq_ai_sdk import Orq
import os

client = Orq(
    api_key=os.environ.get("ORQ_API_KEY", ""),
    server_url="https://my.staging.orq.ai"
)

# Create the agent
agent = client.agents.create(
    request={
        "key": "hr_policy_agent",
        "role": "HR Policy Assistant",
        "description": "Answers questions about company policies and procedures",
        "instructions": "Provide clear, concise answers to HR policy questions. Always cite relevant policy sections.",
        "model": "openai/gpt-4o",
        "path": "agents",
        "settings": {
            "max_iterations": 5,
            "max_execution_time": 300
        }
    }
)

print(f"Agent created: {agent.id}")

Agent created: 01KAGPGY6M8QP8S2Z44S1K4CA3


# Tools

To define custom business logic for HR operations and access add external systems, APIs, and custom functionality you can use Tools such as:

* **HTTP Tools:** Integrate with external APIs (e.g., weather, search, CRM).

* **Function Tools**: Inject custom logic via the OpenAI function-calling schema.

* **Code Tools**: Run small snippets of Python in secure sandboxes for data transformation.

To see a full reference of basic tools see [Tools with Agents](https://docs.orq.ai/docs/agents/tools)

### **Python function calculating PTO on starting date**

In [ ]:
from datetime import datetime, timedelta

def calculate_pto_remaining(start_date, accrual_rate, days_used, accrual_frequency):
    """Calculate remaining PTO days based on starting date and usage."""
    # Parse start date if it's a string
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, '%Y-%m-%d')

    # Calculate days employed
    today = datetime.now()
    days_employed = (today - start_date).days

    # Calculate accrual periods based on frequency
    if accrual_frequency == 'monthly':
        periods = days_employed / 30.44  # Average days per month
    elif accrual_frequency == 'biweekly':
        periods = days_employed / 14
    elif accrual_frequency == 'yearly':
        periods = days_employed / 365.25
    else:  # daily
        periods = days_employed

    # Calculate total PTO accrued
    pto_accrued = periods * accrual_rate

    # Calculate remaining PTO
    pto_remaining = pto_accrued - days_used

    return {
        "pto_remaining": round(pto_remaining, 2),
        "pto_accrued": round(pto_accrued, 2),
        "days_used": days_used,
        "days_employed": days_employed,
        "start_date": start_date.strftime('%Y-%m-%d'),
        "accrual_rate": accrual_rate,
        "accrual_frequency": accrual_frequency
    }

# Define params for execution
params = {}

# Execute with params
result = calculate_pto_remaining(
    params.get('start_date', '2024-01-01'),
    params.get('accrual_rate', 1.25),  # e.g., 1.25 days per month = 15 days/year
    params.get('days_used', 5),
    params.get('accrual_frequency', 'monthly')
)

The function above `calculate_pto_remaining` need to be converted into a paylod string. You can do that from bash using:

```
cat calculate_pto_remaining.py | jq -Rs .
```

### **Agent with tools**

The function `calculate_pto_remaining` is now loaded inside `code_tool` as payload and defined under the variable `PTO_CALCULATOR_CODE`



In [ ]:
from orq_ai_sdk import Orq
import os
from datetime import datetime


# Configuration & Constants
PTO_CALCULATOR_CODE = (
    "from datetime import datetime, timedelta\\n"
    "\\n"
    "def calculate_pto_remaining(start_date, accrual_rate, days_used, accrual_frequency):\\n"
    "    \\\"\\\"\\\"Calculate remaining PTO days based on starting date and usage.\\\"\\\"\\\"\\n"
    "    if isinstance(start_date, str):\\n"
    "        start_date = datetime.strptime(start_date, '%Y-%m-%d')\\n"
    "\\n"
    "    today = datetime.now()\\n"
    "    days_employed = (today - start_date).days\\n"
    "\\n"
    "    if accrual_frequency == 'monthly':\\n"
    "        periods = days_employed / 30.44\\n"
    "    elif accrual_frequency == 'biweekly':\\n"
    "        periods = days_employed / 14\\n"
    "    elif accrual_frequency == 'yearly':\\n"
    "        periods = days_employed / 365.25\\n"
    "    else:\\n"
    "        periods = days_employed\\n"
    "\\n"
    "    pto_accrued = periods * accrual_rate\\n"
    "    pto_remaining = pto_accrued - days_used\\n"
    "\\n"
    "    return {\\n"
    "        \\\"pto_remaining\\\": round(pto_remaining, 2),\\n"
    "        \\\"pto_accrued\\\": round(pto_accrued, 2),\\n"
    "        \\\"days_used\\\": days_used,\\n"
    "        \\\"days_employed\\\": days_employed,\\n"
    "        \\\"start_date\\\": start_date.strftime('%Y-%m-%d'),\\n"
    "        \\\"accrual_rate\\\": accrual_rate,\\n"
    "        \\\"accrual_frequency\\\": accrual_frequency\\n"
    "    }\\n"
    "\\n"
    "params = {k: v for k, v in locals().items() if k in ['start_date', 'accrual_rate', 'days_used', 'accrual_frequency']}\\n"
    "result = calculate_pto_remaining(\\n"
    "    params.get('start_date', '2024-01-01'),\\n"
    "    params.get('accrual_rate', 1.25),\\n"
    "    params.get('days_used', 0),\\n"
    "    params.get('accrual_frequency', 'monthly')\\n"
    ")\\n"
)

print("=" * 60)
print("ORQ.AI HR POLICY AGENT WITH PTO CALCULATOR")
print("=" * 60)
print(f"Timestamp       : {datetime.now().strftime('%Y-%m-%d %H:%M:%S %Z')}")
print(f"Agent Key       : hr_policy_agent")
print(f"Model           : openai/gpt-4o")
print(f"Tool Included   : pto_calculator (code tool)")
print("-" * 60)

with Orq(
    api_key=os.getenv(
        "ORQ_API_KEY",
        ""
    ),  # Truncated for display
    server_url="https://my.staging.orq.ai"
) as orq:

    print("User query:")
    print("'I started on 2024-03-15, I accrue 1.5 days per month, I've used 8 days so far. How much PTO do I have left?'")
    print("-" * 60)

    response = orq.agents.run(request={
        "key": "hr_policy_agent",
        "role": "HR Policy & Benefits Assistant",
        "description": "Answers questions about company policies, benefits, and calculates PTO balances",
        "instructions": """
You are an expert HR assistant. Answer policy questions clearly and cite sources when possible.
When a user asks about PTO balance or vacation days remaining, use the provided PTO calculator tool
to compute the accurate remaining balance based on hire date, accrual rate, and days already used.
        """.strip(),
        "model": "openai/gpt-4o",
        "path": "agents",
        "settings": {
            "max_iterations": 6,
            "max_execution_time": 300,
            "tools": [
                {
                    "key": "pto_calculator",
                    "type": "code",
                    "display_name": "PTO Balance Calculator",
                    "description": "Calculates accrued and remaining PTO days based on hire date, accrual rules, and usage",
                    "requires_approval": False,
                    "code_tool": {
                        "language": "python",
                        "code": PTO_CALCULATOR_CODE,
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "start_date": {
                                    "type": "string",
                                    "format": "date",
                                    "description": "Employee start date (YYYY-MM-DD)",
                                    "default": "2024-01-01"
                                },
                                "accrual_rate": {
                                    "type": "number",
                                    "description": "PTO days accrued per period (e.g., 1.25 for 15 days/year monthly)",
                                    "default": 1.25
                                },
                                "days_used": {
                                    "type": "number",
                                    "description": "Total PTO days already used this year",
                                    "default": 0
                                },
                                "accrual_frequency": {
                                    "type": "string",
                                    "enum": ["daily", "biweekly", "monthly", "yearly"],
                                    "description": "How PTO accrues",
                                    "default": "monthly"
                                }
                            },
                            "required": ["start_date"],
                            "additionalProperties": False
                        }
                    }
                }
            ]
        },
        "message": {
            "role": "user",
            "parts": [
                {
                    "kind": "text",
                    "text": "I started on 2024-03-15, I accrue 1.5 days per month, I've used 8 days so far. How much PTO do I have left?"
                }
            ]
        }
    })

    assert response is not None

    print("Agent execution completed successfully!")
    print("-" * 60)
    print(response)
    print("=" * 60)

ORQ.AI HR POLICY AGENT WITH PTO CALCULATOR
Timestamp       : 2025-11-20 09:15:50 
Agent Key       : hr_policy_agent
Model           : openai/gpt-4o
Tool Included   : pto_calculator (code tool)
------------------------------------------------------------
User query:
'I started on 2024-03-15, I accrue 1.5 days per month, I've used 8 days so far. How much PTO do I have left?'
------------------------------------------------------------
Agent execution completed successfully!
------------------------------------------------------------
id='01KAG8RX7846MGBNK5R1TG5B9D' context_id='01KAG8RX78C24PYFVBZ8DFTKJ8' kind='task' status=RunAgentStatus(state='submitted', timestamp='2025-11-20T09:15:50.969Z', message=None) metadata={'workspace_id': 'e5c3c3dc-483f-42ce-ac82-9e55792d8806', 'trace_id': '01KAG8RX78EMD9Y2J0YJN149BC'}


# Memory and Context

To remember employee preferences and store conversation history we will add in the next step the memory_stores, that would help persist context across sessions. To learn more see [Memory Management with Agents](https://docs.orq.ai/docs/agents/memory-management)

In [ ]:
from orq_ai_sdk import Orq
import os
from datetime import datetime

# ------------------------------------------------------------------
# Configuration & Constants
# ------------------------------------------------------------------
PTO_CALCULATOR_CODE = (
    "from datetime import datetime, timedelta\\n"
    "\\n"
    "def calculate_pto_remaining(start_date, accrual_rate, days_used, accrual_frequency):\\n"
    "    \\\"\\\"\\\"Calculate remaining PTO days based on starting date and usage.\\\"\\\"\\\"\\n"
    "    if isinstance(start_date, str):\\n"
    "        start_date = datetime.strptime(start_date, '%Y-%m-%d')\\n"
    "\\n"
    "    today = datetime.now()\\n"
    "    days_employed = (today - start_date).days\\n"
    "\\n"
    "    if accrual_frequency == 'monthly':\\n"
    "        periods = days_employed / 30.44\\n"
    "    elif accrual_frequency == 'biweekly':\\n"
    "        periods = days_employed / 14\\n"
    "    elif accrual_frequency == 'yearly':\\n"
    "        periods = days_employed / 365.25\\n"
    "    else:\\n"
    "        periods = days_employed\\n"
    "\\n"
    "    pto_accrued = periods * accrual_rate\\n"
    "    pto_remaining = pto_accrued - days_used\\n"
    "\\n"
    "    return {\\n"
    "        \\\"pto_remaining\\\": round(pto_remaining, 2),\\n"
    "        \\\"pto_accrued\\\": round(pto_accrued, 2),\\n"
    "        \\\"days_used\\\": days_used,\\n"
    "        \\\"days_employed\\\": days_employed,\\n"
    "        \\\"start_date\\\": start_date.strftime('%Y-%m-%d'),\\n"
    "        \\\"accrual_rate\\\": accrual_rate,\\n"
    "        \\\"accrual_frequency\\\": accrual_frequency\\n"
    "    }\\n"
    "\\n"
    "params = {k: v for k, v in locals().items() if k in ['start_date', 'accrual_rate', 'days_used', 'accrual_frequency']}\\n"
    "result = calculate_pto_remaining(\\n"
    "    params.get('start_date', '2024-01-01'),\\n"
    "    params.get('accrual_rate', 1.25),\\n"
    "    params.get('days_used', 0),\\n"
    "    params.get('accrual_frequency', 'monthly')\\n"
    ")\\n"
)

print("=" * 70)
print("ORQ.AI HR POLICY AGENT WITH PTO CALCULATOR + MEMORY")
print("=" * 70)
print(f"Timestamp       : {datetime.now().strftime('%Y-%m-%d %H:%M:%S %Z')}")
print(f"Agent Key       : hr_policy_agent")
print(f"Model           : openai/gpt-4o")
print(f"Memory Store    : hr-employee-pto-data")
print(f"Entity ID       : user_{os.getenv('USER_ID', 'John')}")
print("-" * 70)

with Orq(
    api_key=os.getenv(
        "ORQ_API_KEY",
        ""
    ),
    server_url="https://my.staging.orq.ai"
) as orq:

    print("Sending request with memory-enabled agent...")
    print("User query:")
    print("    → 'I started on 2024-03-15, I accrue 1.5 days per month, I've used 8 days so far. How much PTO do I have left?'")
    print("    (Next time you ask, the agent will REMEMBER your details!)")
    print("-" * 70)

    response = orq.agents.run(request={
        "key": "hr_policy_agent",
        "role": "HR Policy & Benefits Assistant (with Memory)",
        "description": "Remembers employee PTO details and answers policy questions",
        "instructions": """
You are a smart HR assistant with memory.
- Always check if the user has previously shared their hire date, accrual rate, or PTO usage using memory tools.
- If they have, recall and use that data instead of asking again.
- When new PTO details are shared, save them to memory for future conversations.
- Use the PTO calculator tool to give precise remaining balance.
        """.strip(),

        # MEMORY CONFIGURATION
        "memory": {
            "entity_id": f"user_{os.getenv('USER_ID', 'JohnSmith')}"
        },
        "memory_stores": [
            "hr-employee-pto-data"   # You must create this store in Orq dashboard first!
        ],

        "model": "openai/gpt-4o",
        "path": "agents",
        "settings": {
            "max_iterations": 8,
            "max_execution_time": 300,
            "tools": [
                # Your custom PTO calculator
                {
                    "key": "pto_calculator",
                    "type": "code",
                    "display_name": "PTO Balance Calculator",
                    "description": "Calculates accrued and remaining PTO days",
                    "requires_approval": False,
                    "code_tool": {
                        "language": "python",
                        "code": PTO_CALCULATOR_CODE,
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "start_date": {"type": "string", "format": "date", "default": "2024-01-01"},
                                "accrual_rate": {"type": "number", "default": 1.25},
                                "days_used": {"type": "number", "default": 0},
                                "accrual_frequency": {"type": "string", "enum": ["daily", "biweekly", "monthly", "yearly"], "default": "monthly"}
                            },
                            "required": ["start_date"],
                            "additionalProperties": False
                        }
                    }
                },
                # Built-in memory tools (required for recall/save)
                {"type": "retrieve_memory_stores"},
                {"type": "query_memory_store"},
                {"type": "write_memory_store"},
                {"type": "delete_memory_document"}
            ]
        },
        "message": {
            "role": "user",
            "parts": [{
                "kind": "text",
                "text": "I started on 2024-03-15, I accrue 1.5 days per month, I've used 8 days so far. How much PTO do I have left?"
            }]
        }
    })

    assert response is not None

    print("Agent execution completed successfully!")
    print("-" * 70)
    print("FINAL RESPONSE:")
    print("-" * 70)
    print(response)
    print("-" * 70)
    print("Memory is now active! Next time you ask about PTO,")
    print("the agent will remember your hire date and usage")
    print("=" * 70)

ORQ.AI HR POLICY AGENT WITH PTO CALCULATOR + MEMORY
Timestamp       : 2025-11-20 09:24:06 
Agent Key       : hr_policy_agent
Model           : openai/gpt-4o
Memory Store    : hr-employee-pto-data
Entity ID       : user_John
----------------------------------------------------------------------
Sending request with memory-enabled agent...
User query:
    → 'I started on 2024-03-15, I accrue 1.5 days per month, I've used 8 days so far. How much PTO do I have left?'
    (Next time you ask, the agent will REMEMBER your details!)
----------------------------------------------------------------------
Agent execution completed successfully!
----------------------------------------------------------------------
FINAL RESPONSE:
----------------------------------------------------------------------
id='01KAG981KXRY7QNW8DVBAW5PYD' context_id='01KAG981KXZPXSCW7N44XDCTDB' kind='task' status=RunAgentStatus(state='submitted', timestamp='2025-11-20T09:24:06.996Z', message=None) metadata={'workspace_id

# Knowledge Base

Make sure that your Agent grounds its responses based on your company policies. To do that we will add a Knowledge Base for contextual accuracy.

### **Create a Knowledge Base**

In [ ]:
import os
from orq_ai_sdk import Orq

with Orq(
    api_key=os.getenv("ORQ_API_KEY"),
    server_url="https://my.staging.orq.ai"  # Your custom staging endpoint
) as orq:
    try:
        res = orq.knowledge.create(
            request={
                "type": "internal",
                "key": "agents",
                "path": "agents",
                "embedding_model": "text-embedding-3-large",
                "top_k": 5,
                "threshold": 0.7,
                "search_type": "hybrid_search",
                "is_private_model": False,
            }
        )
        print("Knowledge base created (my.staging.orq.ai):", res)
    except Exception as e:
        if "already exists" in str(e).lower():
            print('Knowledge base "Customer" already exists on my.staging.orq.ai, fetching...')
            res = orq.knowledge.get(request={"key": "Customer"})
            print("Existing knowledge base (my.staging.orq.ai):", res)
        else:
            raise e

Knowledge base created (my.staging.orq.ai): id='01KAGBYHH3JNP4111Z4GZQJSMQ' created='2025-11-20T10:11:21.252Z' key='agents' domain_id='019a8c33-14ff-7000-9a7e-85c942fc95b3' updated='2025-11-20T10:11:21.252Z' model='openai/text-embedding-3-large' description=None path='agents' created_by_id=None updated_by_id=None type='internal' retrieval_settings=ResponseBodyRetrievalSettings(retrieval_type='hybrid_search', top_k=5, threshold=0.0, rerank_config=Unset(), agentic_rag_config=Unset())


### **Upload a file**

In [ ]:
import os
from orq_ai_sdk import Orq

# Connect to your private staging environment
client = Orq(
    api_key=os.getenv("ORQ_API_KEY"),
    server_url="https://my.staging.orq.ai"
)

# 1. Upload the file (purpose="retrieval" is required for RAG/knowledge use)
file_response = client.files.create(
    file={
        "file_name": "/content/policy.pdf",                # change to your actual file
        "content": open("/content/policy.pdf", "rb"),      # path to your file
    },
    purpose="retrieval"
)

print(f"File uploaded successfully → ID: {file_response.id}")

File uploaded successfully → ID: file_01KAGCHQTRGBP8RB2H3AP98C05


### **Create a datasource**

In [ ]:
import os
from orq_ai_sdk import Orq

client = Orq(
    api_key=os.getenv("ORQ_API_KEY"),
    server_url="https://my.staging.orq.ai"
)

file_id = "file_01KAGCHQTRGBP8RB2H3AP98C05"
knowledge_id = "01KAGBYHH3JNP4111Z4GZQJSMQ"

data_source = client.knowledge.create_datasource(
    knowledge_id=knowledge_id,
    file_id=file_id
)

print(f"Data source created → ID: {data_source.id}")

Data source created → ID: 01KAGCNJ7CE81NXMMRQZ96NV3E


# Multi Agent system

### Company benefits agent

In [ ]:
import os
from orq_ai_sdk import Orq

client = Orq(
    api_key=os.getenv("ORQ_API_KEY"),
    server_url="https://my.staging.orq.ai"
)

# Simple Benefits Specialist — no knowledge base, no errors
benefits_agent = client.agents.create(
    request={
        "key": "hr_benefits_specialist",
        "role": "Benefits Enrollment Expert",
        "description": "Helps employees with health insurance, 401k, PTO, and all benefits questions",
        "instructions": "Answer clearly and kindly. Use current_date tool when needed for eligibility or open enrollment periods.",
        "model": "openai/gpt-4o",
        "path": "agents",                                   # make sure "agents" project exists
        "settings": {
            "max_iterations": 5,
            "max_execution_time": 300,
            "tools": [
                {"type": "current_date"}                    # very useful for benefits questions
            ]
        }
        # ← No knowledge_bases → no 404 error
    }
)

print(f"Benefits agent created successfully!")
print(f"ID : {benefits_agent.id}")
print(f"Key: {benefits_agent.key}")

Benefits agent created successfully!
ID : 01KAGF9YQYZSYY1FK3RRB3VZHM
Key: hr_benefits_specialist


## Orchestrator

In [ ]:
import os
from orq_ai_sdk import Orq

client = Orq(
    api_key=os.getenv("ORQ_API_KEY"),
    server_url="https://my.staging.orq.ai"
)

# === HR ORCHESTRATOR AGENT ===
orchestrator = client.agents.create(
    request={
        "key": "hr_orchestrator",
        "role": "HR Coordinator Assistant",
        "description": "A friendly HR orchestrator that helps employees by coordinating specialized HR sub-agents",
        "path": "agents",  # matches your project structure
        "model": "openai/gpt-4o",
        "instructions": """
You are a helpful and professional HR coordinator. Your job is to understand the employee's question and delegate it to the right specialist agent.

Available specialists:
- hr_benefits_specialist → for health insurance, 401k, open enrollment, benefits eligibility
- hr_pto_calculator_agent → for PTO balance, vacation days, accrual rates, "how much time do I have left?"

Workflow:
1. Use the retrieve_agents tool to confirm available agents.
2. Analyze the user’s question.
3. Call the correct sub-agent using call_sub_agent with a clear, direct instruction.
4. Wait for their response.
5. Return their answer EXACTLY as provided — do not rephrase or add commentary.
6. If unsure, default to hr_benefits_specialist.

Be efficient. Do not overthink. Delegate fast and accurately.
        """.strip(),
        "settings": {
            "max_iterations": 15,
            "max_execution_time": 600,
            "tool_approval_required": "none",
            "tools": [
                {"type": "current_date"},
                {"type": "retrieve_agents"},
                {"type": "call_sub_agent"}
            ]
        },
        "team_of_agents": [
            {
                "key": "hr_benefits_specialist",
                "role": "Benefits Enrollment Expert"
            },
            {
                "key": "hr_policy_agent",
                "role": "PTO & Vacation Balance Calculator"
            }
        ],
        "memory_stores": []
    }
)

print("HR Orchestrator created successfully!")
print(f"Agent ID : {orchestrator.id}")
print(f"Agent Key: {orchestrator.key}")

HR Orchestrator created successfully!
Agent ID : 01KAGGT36ZK06ZB459JAF061XX
Agent Key: hr_orchestrator


Invoking orchestrator

In [ ]:
import os
from orq_ai_sdk import Orq

client = Orq(
    api_key=os.getenv("ORQ_API_KEY"),
    server_url="https://my.staging.orq.ai"
)

# Invoke the orchestrator (persistent agent)
task = client.agents.invoke(
    key="hr_orchestrator",   # ← use .invoke() + key= (not agent_key=)
    message={
        "role": "user",
        "parts": [
            {
                "kind": "text",
                "text": "I started on 2024-03-15, I accrue 1.5 days per month, and I've used 8 days so far. How much PTO do I have left?"
            }
        ]
    }
)

print(f"Orchestrator task started!")
print(f"Task ID: {task.id}")
print(f"Check the result in the ORQ dashboard → https://my.staging.orq.ai/tasks/{task.id}")

Orchestrator task started!
Task ID: 01KAGH5C5RMFCR8T2RNFE6PK3D
Check the result in the ORQ dashboard → https://my.staging.orq.ai/tasks/01KAGH5C5RMFCR8T2RNFE6PK3D
